In [118]:
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

In [119]:
split_date = '2023-01-01'

In [120]:
li = []

li.append(pd.read_csv("../prepared_data/test.csv", index_col=0))

matches = pd.concat(li, axis=0, ignore_index=True)
matches = matches[matches["venue"] == "Home"]
matches.shape

(3335, 130)

In [121]:
matches["year"] = pd.to_datetime(matches["date"]).dt.year
matches["month"] = pd.to_datetime(matches["date"]).dt.month

In [122]:
rf = RandomForestClassifier(n_estimators=10000, n_jobs=-1, random_state=1, max_depth=3)

In [123]:
predictors = ["sota_rolling", "ga_rolling", "opp_rolling", "stp_rolling"]

In [124]:
train = matches[matches["date"] < split_date]
test = matches[matches["date"] > split_date]

In [125]:
rf.fit(train[predictors], train["target"])

RandomForestClassifier(max_depth=3, n_estimators=10000, n_jobs=-1,
                       random_state=1)

In [126]:
predsWinsTrain = rf.predict(train[predictors])

probabilities = rf.predict_proba(test[predictors])
probabilities = probabilities[:, 1]

combinedProbabilities = pd.DataFrame(dict(actual=test["target"], probabilities=probabilities), index=test.index)
combinedProbabilities["date"] = test["date"]

In [127]:
best_threshold = 0
min_diff = float('inf')

thresholds = range(0, 1000, 1)
for threshold in thresholds:
	threshold = threshold / 1000
	predicted = (probabilities >= threshold).astype('int')
	tn, fp, fn, tp = confusion_matrix(combinedProbabilities["actual"], predicted).ravel()
        
	pos = tp + fp
	neg = tn + fn
	diff = abs(neg - (100 / 37.8) * pos)
	
	if diff < min_diff:
		min_diff = diff
		best_threshold = threshold

print(best_threshold)
predicted = (probabilities >= best_threshold).astype('int')
combinedProbabilities["predicted"] = predicted

0.462


In [128]:
print(int(len(combinedProbabilities[(combinedProbabilities["actual"] == 1) & (combinedProbabilities["predicted"] == 1)]) / len(combinedProbabilities[combinedProbabilities["actual"] == 1]) * 1_000_000) / 10_000, "The higher the better, but doesn't have to be high")
print(int(len(combinedProbabilities[(combinedProbabilities["actual"] == 0) & (combinedProbabilities["predicted"] == 1)]) / len(combinedProbabilities[combinedProbabilities["actual"] == 0]) * 1_000_000) / 10_000, "The smaller the better, has to be small")

print(int(len(combinedProbabilities[(combinedProbabilities["actual"] == 1) & (combinedProbabilities["predicted"] == 1)]) / len(combinedProbabilities[combinedProbabilities["predicted"] == 1]) * 1_000_000) / 10_000, "Overall measure of performance in %")

31.8289 The higher the better, but doesn't have to be high
22.6164 The smaller the better, has to be small
56.7796 Overall measure of performance in %


In [129]:
pd.DataFrame(confusion_matrix(combinedProbabilities["actual"], combinedProbabilities["predicted"]))

,0,1
0,349,102
1,287,134


In [130]:
combinedProbabilities.sort_values("date").tail(50)

,actual,probabilities,date,predicted
3082,1,0.481240,2024-01-03,1
3213,1,0.413539,2024-01-03,0
5037,1,0.540196,2024-01-03,1
5371,0,0.460131,2024-01-04,0
4775,1,0.405790,2024-01-04,0
3263,0,0.322457,2024-01-04,0
5372,0,0.416528,2024-01-12,0
1680,0,0.382714,2024-01-12,0
4482,0,0.346673,2024-01-13,0
3264,1,0.306391,2024-01-13,0
